In [59]:
import numpy as np
import tensorflow as tf
import random
from keras.layers import *
import pdb
from keras.models import Model
from functools import partial
from tqdm import tqdm_notebook as tqdm

### Data Loading
from adlframework.processors.general_processors import crop, reshape, pdb_trace
from adlframework.processors.lstm_processors import crop_and_label
from adlframework.processors.midi_processors import midi_to_np, notes_to_classification, make_time_relative
from adlframework.filters.general_filters import min_array_shape, threshold_label
from adlframework.retrievals.BlobLocalCache import BlobLocalCache
from adlframework.datasource import DataSource
from adlframework.dataentity.midi_de import MidiDataEntity

In [2]:
i = Input((100, 3))
bn = BatchNormalization()(i)
lstm_layer = LSTM(100)(bn)
notes = Dense(88)(lstm_layer)
space = Dense(12)(lstm_layer)
duration = Dense(11)(lstm_layer)
notes_prob = Activation('softmax')(notes)
space_prob = Activation('softmax')(space)
duration_prob = Activation('softmax')(duration)
model = Model(i, [space_prob, duration_prob, notes_prob])
model.compile('adam', 'categorical_crossentropy')

In [23]:
def convert_to_one_hot(sample):
    data, label = sample
    ret = []
    for i, v  in enumerate([12, 11, 88]):
        z = np.zeros(v)
        z[label[0][i]] = 1
        ret.append(z)
    return data, ret

## ADLFramework Data

In [27]:
### Controllers
controllers = [partial(threshold_label, labelnames="num_instruments", threshold=1, greater_than=False),
                midi_to_np,
                partial(min_array_shape, min_shape=(101, 4)),
                partial(crop, shape=(101, 3)),
                make_time_relative,
                notes_to_classification,
                partial(crop_and_label, num_rows=1),
                convert_to_one_hot ## Not applied to input data, only label
              ]

### Load Data
base = '/Users/localhost/Desktop/Projects/Working/StudyMuse/local_cache/alex_midiset/v2/'
midi_retrieval = BlobLocalCache(base+'midis/', base+'labels/')
midi_ds = DataSource(midi_retrieval, MidiDataEntity,
                         verbosity=0,
						controllers=controllers,
						backend='madmom',
						batch_size=50)

train_ds, temp = DataSource.split(midi_ds, split_percent=.6) # Train at .6
val_ds, test_ds = DataSource.split(temp, split_percent=.6) # Val at .24, test at .16

INFO:adlframework.datasource:Prefiltering entities


Retrieval not named, so won't be cached.


## Experiment

In [56]:
### Hyperparameters
epochs = 100

In [ ]:
for i in tqdm(range(epochs)):
    batch = train_ds.next()
    data, labels = batch
    labels = [np.array(list(labels[:,i])) for i in range(3)]
    losses = model.train_on_batch(data, labels)
    print(*zip(losses, ['MAE', 'Notes', 'Spaces', 'Durations']))

Widget Javascript not detected.  It may not be installed or enabled properly.


[9.3263245, 2.444927, 2.4018421, 4.4795556]


INFO:adlframework.datasource:Looped the datasource
INFO:adlframework.datasource:Shuffling the datasource


[9.3485603, 2.4758248, 2.3823757, 4.4903593]
[9.2823601, 2.4287977, 2.3712609, 4.4823012]
[9.2568178, 2.4187486, 2.3625295, 4.4755397]
[9.2724934, 2.4488504, 2.3671637, 4.4564786]
[9.2167721, 2.4173594, 2.3296585, 4.4697547]
[9.183672, 2.4111593, 2.3035264, 4.468986]


INFO:adlframework.datasource:Looped the datasource
INFO:adlframework.datasource:Shuffling the datasource


...

[6.192215, 1.5499932, 1.1783724, 3.4638495]
[6.3452415, 1.6906235, 1.1685652, 3.4860528]
[5.9374018, 1.7005795, 0.82843095, 3.408391]
[6.3538599, 1.7999426, 1.0735173, 3.4804001]
[6.3433623, 1.715245, 1.266412, 3.3617053]
[6.4285197, 1.7075016, 0.98297191, 3.7380462]


INFO:adlframework.datasource:Looped the datasource
INFO:adlframework.datasource:Shuffling the datasource


[6.32652, 1.7561562, 1.0818114, 3.4885523]
[6.4669876, 1.7876058, 1.2130295, 3.4663522]
[6.2056026, 1.8000281, 1.052091, 3.3534837]
[6.2745528, 1.5831927, 1.0885662, 3.6027939]
[6.2997904, 1.7907014, 0.92975831, 3.5793307]
[5.8368464, 1.5345997, 0.8509813, 3.4512656]


INFO:adlframework.datasource:Looped the datasource
INFO:adlframework.datasource:Shuffling the datasource


[6.3196797, 1.6365561, 1.2502819, 3.4328418]
